In [29]:
import requests
import pandas as pd
from datetime import datetime
import yfinance as yf
import altair as alt

api_key = 'C6ig1sXku2yKl_XEIvSvc_OWCwB8ILLn'
base_url = 'https://api.polygon.io/v2/aggs/ticker/'

def get_historical_data(ticker, start_date):
    end_date = datetime.now()
    ticker = f'O:{ticker}'
    if isinstance(start_date, datetime):
        start_date = start_date.strftime('%Y-%m-%d')

    url = f"{base_url}{ticker}/range/1/day/{start_date}/{end_date.strftime('%Y-%m-%d')}?apiKey={api_key}"

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'results' in data:
            df = pd.DataFrame(data['results'])
            df['t'] = pd.to_datetime(df['t'], unit='ms')
            df['date'] = df['t'].dt.date
            return df[['date', 'c']]
        else:
            print("No results found in the data.")
            return pd.DataFrame()
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return None

def get_stock_price(symbol, start_date, end_date):
    stock = yf.Ticker(symbol)
    hist = stock.history(start=start_date, end=end_date)
    hist.reset_index(inplace=True)
    hist['date'] = hist['Date'].dt.date
    hist.rename(columns={'Close': 'stock_close_price'}, inplace=True)
    hist['stock'] = hist['stock_close_price'].round(2)
    return hist[['date', 'stock']]

def calculate_pnl(call_action, put_action, NC, C_0, C_t, NP, P_0, P_t, effective_delta, trade_price, current_price):
    if call_action == "sell" and put_action == "sell":
        return (NC * (C_0 - C_t) + NP * (P_0 - P_t) + effective_delta * (current_price - trade_price)) * 100
    elif call_action == "sell" and put_action == "buy":
        return (NC * (C_0 - C_t) + NP * (P_t - P_0) + effective_delta * (current_price - trade_price)) * 100
    elif call_action == "buy" and put_action == "sell":
        return (NC * (C_t - C_0) + NP * (P_0 - P_t) + effective_delta * (current_price - trade_price)) * 100
    elif call_action == "buy" and put_action == "buy":
        return (NC * (C_t - C_0) + NP * (P_t - P_0) + effective_delta * (current_price - trade_price)) * 100
    else:
        return 0

def data(call_ticker, put_ticker, trade_date):
    call_data = get_historical_data(call_ticker, trade_date)
    put_data = get_historical_data(put_ticker, trade_date)
    
    if call_data.empty or put_data.empty:
        return pd.DataFrame()
    
    call_data.rename(columns={'c': 'call_close_price'}, inplace=True)
    put_data.rename(columns={'c': 'put_close_price'}, inplace=True)
    
    data = pd.merge(call_data, put_data, on='date', how='inner')
    
    symbol = call_ticker[:next((i for i, char in enumerate(call_ticker) if char.isdigit()), None)]
    length = len(symbol)
    date = call_ticker[length:length + 6]
    expire_date = f"20{date[:2]}-{date[2:4]}-{date[4:]}"
    stock_data = get_stock_price(symbol, trade_date, expire_date)

    data = pd.merge(data, stock_data, on='date', how='inner')
    
    return data

def get_pnl(call_ticker, put_ticker, trade_date, stock_trade_price, effective_delta, call_action, NC, C_0, put_action, NP, P_0):
    pnl_data = data(call_ticker, put_ticker, trade_date)
    if pnl_data.empty:
        print("No data available for the given parameters.")
        return pd.DataFrame()

    pnl_data['pnl'] = pnl_data.apply(lambda x: calculate_pnl(call_action, put_action, NC, C_0, x['call_close_price'], NP, P_0, x['put_close_price'], effective_delta, stock_trade_price, x['stock']), axis=1)
    return pnl_data

def plot_pnl(pnl_data, call_action, num_call_contracts, put_action, num_put_contracts):
    if pnl_data.empty:
        print("No data to plot.")
        return
    
    pnl_data['color'] = pnl_data['pnl'].apply(lambda x: '#bd1414' if x < 0 else '#007560')
    pnl_data['date'] = pd.to_datetime(pnl_data['date']).dt.strftime('%m-%d')

    # Create a line plot with points, ensuring the line is black
    line = alt.Chart(pnl_data).mark_line(
        color='black',  # Set line color to black
        size=2          # Set line thickness
    ).encode(
        x=alt.X('date:N', title='Date'),
        y=alt.Y('pnl:Q', title='Profit and Loss'),
        tooltip=['date:N', 'pnl:Q', 'call_close_price:Q', 'put_close_price:Q', 'stock:Q']
    )

    points = alt.Chart(pnl_data).mark_point(
        color='black',  # Ensure points are black
        size=100        # Increased point size
    ).encode(
        x='date:N',
        y='pnl:Q',
        fill=alt.Color('color:N', scale=None)  # Fill points based on pnl value
    )

    # Define a rule for the horizontal line at PNL = 0
    rule = alt.Chart(pnl_data).mark_rule(color='black').encode(
        y=alt.Y('a:Q', axis=alt.Axis(title='')),
    ).transform_calculate(
        a='0'  # Constant value for 0
    )

    # Configure the chart with a grid background like Seaborn's style
    chart = (line + points + rule).properties(
        title=f'{call_action.capitalize()} {num_call_contracts} Call(s) & {put_action.capitalize()} {num_put_contracts} Put(s)',
        width=800,
        height=400
    ).configure_axis(
        grid=True,                  # Enable grid
        gridColor='gray',           # Grid color
        gridDash=[6, 3]             # Dash pattern like Seaborn
    ).configure_view(
        strokeWidth=0               # Remove border around chart
    )
    
    # add zoom in and out
    chart = chart.interactive()
    
    # Configure the tooltip to show all encoding channels
    chart = chart.configure_mark(tooltip=alt.TooltipContent('encoding'))

    chart.show()

def main(call_ticker, put_ticker, trade_date, stock_trade_price, effective_delta, call_action, NC, C_0, put_action, NP, P_0):
    pnl_data = get_pnl(call_ticker, put_ticker, trade_date, stock_trade_price, effective_delta, call_action, NC, C_0, put_action, NP, P_0)
    plot_pnl(pnl_data, call_action, NC, put_action, NP)



In [30]:
main(call_ticker='AAPL240621C00180000', put_ticker='AAPL240621P00180000', trade_date='2024-05-01', stock_trade_price=200, effective_delta=0.3, call_action='sell', NC=1, C_0=11, put_action='sell', NP=1, P_0=6)


alt.LayerChart(...)